# **Semanas 09: Pré-Processamento de Dados II**
## **Atividade Semanal**

> Esta é atividade semanal sugerida. Você deverá preenchê-la e enviá-la dentro do prazo (até o fechamento da semana corrente).

In [53]:
# NAO ALTERE OS COMANDOS ABAIXO
import gdown
import pandas as pd

url = 'https://drive.google.com/uc?id=1DwqWeEuzjuMn_WAmz7U3Uo49I1zeaZyg'
output = 'MFGEmployees4.xlsx'
gdown.download(url, output, quiet=True)

dados_funcionarios = pd.read_excel('/content/MFGEmployees4.xlsx')

> A partir dos dados organizados da atividade passada, vamos aplicar os conhecimentos aprendidos sobre Engenharia de atributos.

In [54]:
# Use a resolucao da atividade passada para seguir com os dados organizados:
df_dados_func = dados_funcionarios.drop_duplicates() #removendo duplicadas
df_dados_func = df_dados_func.dropna()#removendo valores ausentes
df_dados_func = df_dados_func.drop(['id','sobrenome','nome'], axis=1) #removendo as 3 variáveis
df_dados_func.describe() #removendo valores descrepantes ou suspeitos

#filtrando todos os valores em que: a  IDADE seja maior que 14 anos e menor que 80 anos, e a DURAÇÃO DE SERVIÇO menor que  44 horas
df_dados_func = df_dados_func[(df_dados_func['idade']>14) &
                              (df_dados_func['idade']<80) &
                              (df_dados_func['duracao_servico']<=44)]
df_dados_func.describe()


display(df_dados_func)

,sexo,cidade,cargo,localizacao,divisao,idade,duracao_servico,horas_ausencia,unidade
0,F,Burnaby,Baker,Burnaby,Lojas,33,6.018478,36.577306,Lojas
1,M,Courtenay,Baker,Nanaimo,Lojas,41,5.532445,30.165072,Lojas
2,M,Richmond,Baker,Richmond,Lojas,49,4.389973,83.807798,Lojas
3,F,Victoria,Baker,Victoria,Lojas,45,3.081736,70.020165,Lojas
4,M,New Westminster,Baker,New Westminster,Lojas,36,3.619091,0.000000,Lojas
...,...,...,...,...,...,...,...,...,...
8336,F,Vancouver,Dairy Person,Vancouver,Lojas,47,7.819204,151.285105,Lojas
8337,F,Langley,Cashier,Langley,Lojas,47,4.838288,93.665111,Lojas
8338,M,Montney,Cashier,Fort St John,Lojas,35,2.427274,0.000000,Lojas
8340,F,Vancouver,Dairy Person,Vancouver,Lojas,44,6.154837,60.321917,Lojas


###Questão 1

Defina a variável "alvo" como `horas_ausencia` e "atributos" como as demais variáveis.

In [55]:
# Questao 1
# Questao 1
alvo= "horas_ausencia"
atributos=['sexo','cidade','cargo','localizacao','divisao','idade','unidade','duracao_servico']

###Questão 2

Crie as bases `X` e `y` com as variaveis adequadas.

In [56]:
# Questao 2
from sklearn.preprocessing import StandardScaler
y = df_dados_func[alvo].to_frame()
x =df_dados_func[atributos]
display(y.head())

,horas_ausencia
0,36.577306
1,30.165072
2,83.807798
3,70.020165
4,0.000000


In [57]:
display(x.head())

,sexo,cidade,cargo,localizacao,divisao,idade,unidade,duracao_servico
0,F,Burnaby,Baker,Burnaby,Lojas,33,Lojas,6.018478
1,M,Courtenay,Baker,Nanaimo,Lojas,41,Lojas,5.532445
2,M,Richmond,Baker,Richmond,Lojas,49,Lojas,4.389973
3,F,Victoria,Baker,Victoria,Lojas,45,Lojas,3.081736
4,M,New Westminster,Baker,New Westminster,Lojas,36,Lojas,3.619091


###Questão 3

Separando os atributos em 2 classes: numéricos e categóricos

In [58]:
# Questao 3
atrib_num=['idade','duracao_servico']
atrib_cat=['sexo','cidade','cargo','localizacao','divisao','unidade']

###Questão 4

Faça o `pipeline` de forma adequada aplicando as transformações necessárias para cada classe.

Dica: Não se esqueça de carregar as bibliotecas!

In [72]:
# Questao 4
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.compose import TransformedTargetRegressor
from sklearn.preprocessing import OneHotEncoder
from sklearn.decomposition import PCA
from sklearn.preprocessing import PowerTransformer, StandardScaler


# Encadeando (em sequencia) as 3 transformacoes para atributos numéricos
# Removing PCA to keep the number of features consistent
num_pipe = Pipeline([
    ('padronizacao', StandardScaler()),
    ('transformacao', PowerTransformer()),
    #('componentes', PCA()) # Removing this line
])

# Sequenciando as transformacoes necessarias
atributos_transf = ColumnTransformer([
    # Transformacoes - Atributos numericos
    ('Atributos numericos', num_pipe, atrib_num),
    # Transformacoes - Atributos categoricos
    ('Atributos categoricos', OneHotEncoder(dtype='int', drop='first',sparse_output=False), atrib_cat)], #adding sparse=False
    remainder='drop',
    verbose_feature_names_out=False
)

In [73]:
# Aplicando internamente a transformacao
atributos_transf.fit(x)

ColumnTransformer(transformers=[('Atributos numericos',
                                 Pipeline(steps=[('padronizacao',
                                                  StandardScaler()),
                                                 ('transformacao',
                                                  PowerTransformer())]),
                                 ['idade', 'duracao_servico']),
                                ('Atributos categoricos',
                                 OneHotEncoder(drop='first', dtype='int',
                                               sparse_output=False),
                                 ['sexo', 'cidade', 'cargo', 'localizacao',
                                  'divisao', 'unidade'])],
                  verbose_feature_names_out=False)

In [74]:
# Tendo uma ideia de como fica o resultado
# (exatamente o mesmo que obtemos anteriormente)
display(pd.DataFrame(atributos_transf.transform(x), columns=atributos_transf.get_feature_names_out()).head())

,idade,duracao_servico,sexo_M,cidade_Agassiz,cidade_Aiyansh,cidade_Aldergrove,cidade_Alexis Creek,cidade_Alkali Lake,cidade_Armstrong,cidade_Ashcroft,...,localizacao_Victoria,localizacao_West Vancouver,localizacao_White Rock,localizacao_Williams Lake,divisao_Executiva,divisao_InfoTech,divisao_Legal,divisao_Lojas,divisao_RH,unidade_Lojas
0,-0.981325,0.792475,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0
1,-0.155190,0.587483,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0
2,0.657260,-0.005454,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0
3,0.252781,-0.928956,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0
4,-0.670230,-0.518563,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0


In [75]:
# Sequenciando as transformacoes necessarias
# para um alvo numérico
alvo_num = TransformedTargetRegressor(
    regressor=None, # Um algoritmo ML entrara aqui!
    transformer=num_pipe
)

In [76]:
# Sequenciando as transformacoes necessarias
# para um alvo categórico
alvo_cat = Pipeline(steps=[
   # Um algoritmo ML entrara aqui!
   ])

In [77]:
# Pipeline: Alvo numerico
pipe_num = Pipeline(steps=[
   ('atributos', atributos_transf),
   ('alvo', alvo_num)])

In [78]:
display(pipe_num)

Pipeline(steps=[('atributos',
                 ColumnTransformer(transformers=[('Atributos numericos',
                                                  Pipeline(steps=[('padronizacao',
                                                                   StandardScaler()),
                                                                  ('transformacao',
                                                                   PowerTransformer())]),
                                                  ['idade', 'duracao_servico']),
                                                 ('Atributos categoricos',
                                                  OneHotEncoder(drop='first',
                                                                dtype='int',
                                                                sparse_output=False),
                                                  ['sexo', 'cidade', 'cargo',
                                                   'localizacao', 'divisao',
                                                   'unidade'])],
                                   verbose_feature_names_out=False)),
                ('alvo',
                 TransformedTargetRegressor(transformer=Pipeline(steps=[('padronizacao',
                                                                         StandardScaler()),
                                                                        ('transformacao',
                                                                         PowerTransformer())])))])

In [79]:
# Pipeline: Alvo categorico
pipe_cat = Pipeline(steps=[
   ('atributos', atributos_transf),
   ('alvo', alvo_cat)])

In [80]:
display(pipe_cat)

Pipeline(steps=[('atributos',
                 ColumnTransformer(transformers=[('Atributos numericos',
                                                  Pipeline(steps=[('padronizacao',
                                                                   StandardScaler()),
                                                                  ('transformacao',
                                                                   PowerTransformer())]),
                                                  ['idade', 'duracao_servico']),
                                                 ('Atributos categoricos',
                                                  OneHotEncoder(drop='first',
                                                                dtype='int',
                                                                sparse_output=False),
                                                  ['sexo', 'cidade', 'cargo',
                                                   'localizacao', 'divisao',
                                                   'unidade'])],
                                   verbose_feature_names_out=False)),
                ('alvo', Pipeline(steps=[]))])